RNN - Erro dos pesos computados e usado somente durante a iteração

In [1]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
import operator as op
import matplotlib.pyplot as plt
import math
import time
from Utils99 import *
from CLOUD import *
#from TEDA import *
from TEDA2 import *
from RLS_VDF import *
from RLS_LOG import *
from RLS_EXP import *
from Utils_graph import *
import optuna
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_pareto_front
from optuna.importance import get_param_importances
from optuna.samplers import RandomSampler
import sys
from TedaGraphs import *


def stop_trial(study, trial):
    global last_improvement_time
    current_time = time.time()

    if study.best_value < best_score[0]:
        best_score[0] = study.best_value
        last_improvement_time = current_time
        
    elif (current_time - last_improvement_time) > 3600:
        print("\n⚠️ Interrompendo: 20 minutos sem melhoria no score.")
        study.stop()

last_improvement_time = time.time()
best_score = [float('inf')]


c:\Users\claud\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning:

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html



Bearing1_1 - Erro: 226.23188206798926 parameters:  [17, 14, 1e-05, 0.0001, 1e-05, 12.0, 1.6, 0.3, 0.98]

In [2]:
optuna.visualization.plot_parallel_coordinate(study)

NameError: name 'study' is not defined

In [ ]:
optuna.visualization.plot_slice(study, params=['N1','N2','N3','fator','m','nI','nR','tau'])

In [ ]:
j=4
brngs = ['Bearing1_1.csv','Bearing1_2.csv','Bearing1_3.csv','Bearing2_2.csv','Bearing2_4.csv']
names = ['B1_1','B1_2','B1_3','B2_2','B2_4']
brng = brngs[j]
p_rs, p_hi = 'RS/' + brng, 'HI/' + brng
df1, df2 = pd.read_csv(p_rs), pd.read_csv(p_hi)
RS = df1['RS'].values
HI = df2['PC1'].values
def objective(trial):
    nI = trial.suggest_int('nI', 5, 5) 
    nR = trial.suggest_int('nR', 19, 26) 
    N1 = trial.suggest_float('N1', 1e-5, 1e-2, log=True)
    N2 = trial.suggest_float('N2', 1e-5, 1e-2, log=True)
    N3 = trial.suggest_float('N3', 1e-5, 1e-2, log=True)
    tau = trial.suggest_int('tau', 1, 25)
    m = trial.suggest_float('m', 1.24, 1.38, step=0.02)
    fator = trial.suggest_float('fator', 0.95, 0.99, step=0.01)

    X = np.array([RS[i:i+nI] for i in range(len(RS)-(nI-1))])
    Y = np.array([HI[i:i+nI] for i in range(len(HI)-(nI-1))])
    xS = X[1:]
    yS = Y[:-1]
    zS = Y[1:]

    teda = TEDARegressor(nI, nR, nI, N1, N2, N3, tau, m, 0.3, fator, ep=0.1, wta=False)

    for i in range(len(xS)):
        x, y, z = xS[i], yS[i], zS[i]
        teda.run(x)
        teda.adapt(y, z)
        teda.RUL_single(y)

        if i % 10 == 0:  # checkpoint de avaliação
            trial.report(teda.MAPE(), step=i)
            if trial.should_prune():
                raise optuna.TrialPruned()

    return teda.MAPE()

study = optuna.create_study(direction="minimize",storage="sqlite:///"+'Test_'+names[j]+".db",study_name=names[j]+'P3',
                            load_if_exists=True,
                            sampler=optuna.samplers.CmaEsSampler(),
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=1000,  n_warmup_steps=20, interval_steps=10)
        ,)

study.optimize(objective, n_trials=5000,)

best_params = study.best_params
Mp = list(best_params.values())
print(brng[:-4], '- Erro:', study.best_value, 'parameters: ', Mp)

Bearing1_1 - Erro: 518.241307569085 parameters:  [5, 24, 4.6575317621958215e-05, 0.00010177377188548301, 0.0.00023187938521582915, 15, 1.24, 0.99]\
Bearing1_2 - Erro: 643.935757324285 parameters:  [5, 18, 0.0011587259603807196, 0.00022711225579341771, 0.0005003578521059077, 7, 1.34, 0.98]\
Bearing1_3 - Erro: 114.10151891386829 parameters:  [5, 18, 0.00029985760409049995, 3.251063081814307e-05, 0.0001230888199580327, 12, 1.34, 0.96]\
Bearing2_2 - Erro: 577.4032342797062 parameters:  [5, 24, 0.0006826086663470173, 4.040452823742237e-05, 0.0007316593705663688, 18, 1.3599999999999999, 0.96]\
Bearing2_4 - Erro: 22.417090598923973 parameters:  [5, 15, 0.004268234559619661, 0.0002719800427134719, 0.0014263483063038727, 23, 1.3, 0.99]

In [ ]:
p_rs, p_hi = 'RS/'+brng, 'HI/'+brng
df1, df2 = pd.read_csv(p_rs), pd.read_csv(p_hi)
RS = df1['RS'].values
HI = df2['PC1'].values

nI,nR,N1,N2,N3,tau,m,fator = Mp
X = np.array([RS[i:i+nI] for i in range(len(RS)-(nI-1))])
Y = np.array([HI[i:i+nI] for i in range(len(HI)-(nI-1))])
xS = X[1:]
yS = Y[:-1]
zS = Y[1:]
teda = TEDARegressor(nI,nR,nI,N1,N2,N3,tau,m,0.3,fator,ep=0.077,wta=False)

for i in range(len(xS)):
    x,y,z = xS[i],yS[i],zS[i]
    teda.run(x)
    teda.adapt(y,z)
    teda.RUL_uncertainty(y)

print('granules qtd:',teda.g,'granules total:',len(teda.c) ,'MAPE:',teda.MAPE())
name = brng
plot_RUL_CI(teda,startX=10,endX=165,startY=-1,endY=160,lw1=1.5,lw2=2,name=None)
#plot_HI(teda,name=name)
plot_DSI(teda,name=name)


In [ ]:
j=1
brngs = ['Bearing1_1.csv','Bearing1_2.csv','Bearing1_3.csv','Bearing2_2.csv','Bearing2_4.csv']
names = ['B1_1','B1_2','B1_3','B2_2','B2_4']
brng = brngs[j]
p_rs, p_hi = 'RS/' + brng, 'HI/' + brng
df1, df2 = pd.read_csv(p_rs), pd.read_csv(p_hi)
RS = df1['RS'].values
HI = df2['PC1'].values
def objective(trial):
    nI = trial.suggest_int('nI', 5, 5) 
    nR = trial.suggest_int('nR', 19, 26) 
    N1 = trial.suggest_int('N1', 1, 5) 
    N2 = trial.suggest_int('N2', 1, 5) 
    N3 = trial.suggest_int('N3', 1, 5) 
    tau = trial.suggest_int('tau', 1, 25)
    m = trial.suggest_float('m', 1.24, 1.38, step=0.02)
    fator = trial.suggest_float('fator', 0.95, 0.99, step=0.01)

    X = np.array([RS[i:i+nI] for i in range(len(RS)-(nI-1))])
    Y = np.array([HI[i:i+nI] for i in range(len(HI)-(nI-1))])
    xS = X[1:]
    yS = Y[:-1]
    zS = Y[1:]

    teda = TEDARegressor(nI, nR, nI, N1, N2, N3, tau, m, 0.3, fator, ep=0.1, wta=False)

    for i in range(len(xS)):
        x, y, z = xS[i], yS[i], zS[i]
        teda.run(x)
        teda.adapt(y, z)
        teda.RUL_single(y)

        if i % 10 == 0:  # checkpoint de avaliação
            trial.report(teda.MAPE(), step=i)
            if trial.should_prune():
                raise optuna.TrialPruned()

    return teda.MAPE()

study = optuna.create_study(direction="minimize",storage="sqlite:///"+'2Test_'+names[j]+".db",study_name=names[j]+'P3',
                            load_if_exists=True,
                            sampler=optuna.samplers.CmaEsSampler(),
                            pruner=optuna.pruners.MedianPruner(n_startup_trials=1000,  n_warmup_steps=20, interval_steps=10)
        ,)

study.optimize(objective, n_trials=5000,)

best_params = study.best_params
Mp = list(best_params.values())
print(brng[:-4], '- Erro:', study.best_value, 'parameters: ', Mp)

0.8387096774193549

In [ ]:
brng = 'Bearing2_4.csv'
p_rs, p_hi = '12_RS/' + brng, '08_HI/' + brng
df1, df2 = pd.read_csv(p_rs), pd.read_csv(p_hi)
RS = df1['RS'].values
HI = df2['PC1'].values

# Variáveis para monitoramento
last_improvement_time = time.time()
best_score = [float('inf')]  # Lista mutável para uso dentro do callback

# Função objetivo
def objective(trial):

    Mp=set_define(brng)
    nI,nR,N1,N2,N3,tau,m,fator = Mp
    ep = trial.suggest_float('ep', 0.001, 0.1, step=0.001)

    X = np.array([RS[i:i+nI] for i in range(len(RS)-(nI-1))])
    Y = np.array([HI[i:i+nI] for i in range(len(HI)-(nI-1))])
    xS = X[1:]
    yS = Y[:-1]
    zS = Y[1:]

    teda = TEDARegressor(nI, nR, nI, N1, N2, N3, tau, m, 0.3, fator, ep, wta=False)

    for i in range(len(xS)):
        x, y, z = xS[i], yS[i], zS[i]
        teda.run(x)
        teda.adapt(y, z)
        #teda.RUL_single(y)
        teda.RUL_uncertainty(y)

    score = teda.specificity()*teda.coverage()
    return score

# Callback de early stopping por tempo sem melhoria
def stop_if_no_improvement_for_20_minutes(study, trial):
    global last_improvement_time
    current_time = time.time()

    # Se nova melhoria for encontrada
    if study.best_value < best_score[0]:
        best_score[0] = study.best_value
        last_improvement_time = current_time
    # Verifica se já se passaram 20 minutos (1200 segundos) sem melhoria
    elif (current_time - last_improvement_time) > 1200:
        print("\n⚠️ Interrompendo: 20 minutos sem melhoria no score.")
        study.stop()

# Execução da otimização
study = optuna.create_study(direction="maximize",storage="sqlite:///"+brng[:-4]+".db",study_name=brng[:-4]+'_R1',load_if_exists=True)
study.optimize(objective, n_trials=1000, callbacks=[stop_if_no_improvement_for_20_minutes])
#importances = get_param_importances(study)

# Exibição dos melhores parâmetros
best_params = study.best_params
Mp = list(best_params.values())
print("\n🔹 Melhor Configuração Encontrada:")
print(brng[:-4], '- Erro:', study.best_value, 'parameters: ', Mp)

Bearing1_1 - Erro: 0.5505970002940888 parameters:  [0.08800000000000001]\
Bearing1_2 - Erro: 0.15093640762089958 parameters:  [0.07100000000000001]\
Bearing1_3 - Erro: 0.3281329975636222 parameters:  [0.095]\
Bearing2_2 - Erro: 0.32701118568232657 parameters:  [0.099]\
Bearing2_4 - Erro: 0.7180020811654525 parameters:  [0.027000000000000003]\

